# 2. Programmierprojekt: Local Search

## $n$-Damen Problem

Wir modellieren das $n$-Damen Problem wie folgt: Jeder Zustand im Suchraum ist eine Permutation des Vektors $(0, \dotsc, n-1)$. Damit sind die Aktionen die von einem Zustand möglich sind Vertauschungen von Zahlen.

In [2]:
import random

def generate_random_conf(n: int) -> list:
  # Generate random permutation of (0,...,n-1)
  conf = [i for i in range(n)]
  random.shuffle(conf)
  return conf

def swap(conf, i, j):
  # Swap positions i,j inplace
  conf[i], conf[j] = conf[j], conf[i]

In [3]:
conf = generate_random_conf(8)
swap(conf, 3, 5)
conf

[6, 2, 4, 0, 3, 1, 5, 7]

Implentieren Sie die folgende Funktion `conflicts`, welche für jede Dame die Anzahl der Bedrohungen aufsummiert. Überlegen Sie, welche Art von Bedrohungen durch die Modellierung überhaupt möglich sind.

In [4]:
def conflicts(queens) -> int:
  """ Heuristic that indicates number of beatable queens on the right"""
  
  conflicts = 0
  for i in range(len(queens)):
    # For better initialization
    if queens[i] == float("-inf"):
        continue
    for j in range(i + 1, len(queens)):
        conflicts += int(queens[i] == queens[j] or queens[i] + j - i == queens[j] or queens[i] - j + i == queens[j])
  return conflicts

In [5]:
conf1 = [0, 1, 2, 3, 4, 5, 6, 7] # conflicts = 28
conf2 = [17, 22, 11, 5, 2, 6, 12, 15, 21, 19, 10, 8, 0, 3, 1, 20, 23, 9, 14, 18, 13, 24, 16, 4, 7] # conflicts = 0
conf3 = [15, 12, 9, 16, 0, 18, 6, 11, 19, 7, 13, 3, 10, 4, 1, 2, 8, 5, 14, 17] # conflicts = 12


assert conflicts(conf1) == 28
assert conflicts(conf2) == 0
assert conflicts(conf3) == 12

Nutzen sie die obige Funktion als Heuristik für den A*-Algorithmus. Testen sie bis zu welchem $n$ der Algorithmus eine Lösung in unter zwei Minuten findet. Starten Sie dabei immer von einer zufälligen Startkonfiguration.

In [18]:
from queue import PriorityQueue


def queens_a_star(n: int) -> list[int]:
    """ Find solution with A*; not feasible for n > ? """
    
    conf = generate_random_conf(n)
    
    queue = PriorityQueue()
    # stores f, h, g, configuration
    queue.put((conflicts(conf), conflicts(conf), 0, conf))
    visited = []
    
    while not queue.empty():
        entry = queue.get()
        g_score, state = entry[2], entry[3]
        visited.append(state)

        for successor in successors(state):
            if not conflicts(successor):
                return successor
            if successor in visited:
                continue
                
            new_g_score = g_score + 1
            h = conflicts(successor)
            f_score = h + new_g_score
            queue.put((f_score, h, new_g_score, successor))
            visited.append(successor)

    return [-1]


def successors(queens) -> list[list[int]]:
    """ Returns list of successor states; action is swapping """
    successors = []
    for i in range(len(queens)):
        for j in range(len(queens)):
            if i == j:
                continue
            copy = queens.copy()
            swap(copy, i, j)
            successors.append(copy)
    return successors

# TODO find 2 min mark
    

[35, 16, 12, 29, 11, 2, 15, 38, 20, 4, 6, 19, 26, 18, 34, 32, 3, 7, 39, 8, 14, 5, 1, 27, 37, 31, 25, 10, 21, 0, 28, 23, 33, 13, 9, 17, 36, 22, 24, 30]


Wir wollen im Folgenden einen Local-Search Ansatz nutzen, um das Problem zu lösen. Implementieren Sie nun den Hill-Climbing Algorithmus aus der Vorlesung. Der Algorithmus sollte zusätzlich maximal $k$ Seitwärtszüge erlauben.

In [31]:
def queens_hill_climb(n, k=0) -> tuple[list[int], int]:
  """ Searches solution by hill-climbing """
  conf = generate_random_conf(n)
  conflicts_amount = conflicts(conf)
  improvement, side_steps = 1, 0
  visited = []
  
  while side_steps <= k  and conflicts_amount != 0:
      visited.append(conf) # TODO check if not in
      new_conf, new_conflict_amount = successors(conf, visited)
      
      improvement = conflicts_amount - new_conflict_amount
      
      if improvement:
          side_steps = 0
      else:
          side_steps += 1
          
      conf, conflicts_amount = new_conf, new_conflict_amount
      
  return conf, conflicts(conf)


def successors(queens, visited) -> list[list[int]]:
    """ Returns best successor"""
    best_successor = (queens, conflicts(queens))
    for i in range(len(queens)):
        for j in range(len(queens)):
            if i == j:
                continue
            copy = queens.copy()
            swap(copy, i, j)
            best_successor = (copy, conflicts(copy)) if (conflicts(copy) <= best_successor[1] and copy not in visited) else best_successor
    return best_successor


([4, 2, 7, 5, 1, 3, 0, 6], 1)


Evaluieren Sie den Algorithmus empirisch. Testen sie für verschiedene $n$ und $k$, wie groß die Erfolgsrate des Algorithmus ist. Überlegen Sie sich eine geeignete Visualisierung ihrer Ergebnisse.

In [32]:
n_samples = 200
k = [0, 5, 10, 20, 50, 100]

amount, hits = 0, 0
for n in range(8, n_samples):
    for i in k:
        amount += 1
        hits += int(not queens_hill_climb(n, i)[1])
print("Success rate: " + str((hits / amount) * 100) +"%")

Success rate: 80.3030303030303%


Implementieren sie nun Hill-Climbing mit Random-Restarts, um immer optimale Lösungen zu finden. Berechnen Sie auch, wie viele Restarts nötig waren, um eine optimale Lösung zu finden. Bis zu welchem $n$ können sie mit diesem Ansatz in weniger als zwei Minuten Lösungen finden? (Probieren sie auch verschiedene $k$)

In [61]:
def queens_random_restart(n, k=0) -> tuple[list[int], int]:
  restart = 0
  conf, conflict_amount = queens_hill_climb(n, k)
  while conflict_amount:
      restart += 1
      conf, conflict_amount = queens_hill_climb(n, k)
  return conf, restart

# TODO find 2 min mark

Eine weitere Verbesserung kann erreicht werden, in dem die initiale Konfiguration nicht rein zufällig gewählt wird. Es kann versucht werden, anfangs eine Konfiguration zu finden, in der möglichst viele Damen bereits konfliktfrei gesetzt sind. Die restlichen Damen sollten dann mit möglichst wenigen Konflikten gesetzt werden.

Implementieren Sie darauf basierend einen verbesserten Generator für die Startkonfiguration und testen Sie, ob damit noch größere Probleminstanzen gelöst werden können.

In [59]:
from random import randrange


def better_initial_configuration(n: int) -> list[int]:
    
    conf = [float("-inf")] * n
    conf[0] = randrange(n)
    for i in range(1, n):
        copy = conf.copy()
        min_conflicts = 0, 0
        for j in range(n):
            copy[i] = j
            if conflicts(copy) <= min_conflicts[1]: # TODO minimal field abduxation
                min_conflicts = j, conflicts(copy)
        conf[i] = min_conflicts[0]
        
    return conf, conflicts(conf)
    

print(better_initial_configuration(10))
conf = generate_random_conf(10)
print(conf, conflicts(conf))

([2, 9, 7, 4, 8, 3, 5, 0, 0, 0], 6)
[2, 4, 6, 9, 0, 7, 5, 1, 3, 8] 5


In [22]:
n_samples = 200
k = [0, 5, 10, 20, 50, 100]

amount, hits = 0, 0
for n in range(8, n_samples):
    for i in k:
        amount += 1
        hits += int(not better_initial_configuration(n)[1])
print("Success rate: " + str((hits / amount) * 100) +"%")

KeyboardInterrupt: 



---



## Travelling-Salesman Problem

In diesem Teil soll das TSP mithilfe von Local Search approximiert werden. Für diese Aufgabe betrachten wir ausschließlich das symmetrische TSP, bei dem die Kanten der Graphen ungerichtet sind. Es gibt folglich für einen Graph nur eine Tour.

Wir verwenden zur Darstellung der Graphen das Paket `networkx` (https://networkx.org/documentation/latest/). Außerdem das Paket `tsplib95` (https://tsplib95.readthedocs.io/en/stable/index.html) um die Algorithmen mit Benchmarks zu testen.

In [28]:
import networkx as nx
import tsplib95

Implementieren Sie die Funktion `import_benchmarks`, welche die verschiedenen TSP Instanzen zusammen mit den Lösungen aus dem Ordner `tsp_benchmarks` importiert und eine Liste aus Tupeln der Form `(G, optimal_solution)` zurückgibt, bestehend aus dem Graphen als `networkx`-Objekt und dem Gewicht der optimalen Lösung.

In [29]:
def import_benchmarks(path='./tsp_benchmarks') -> list[tuple[nx.Graph, int]]:
  pass

Implementieren Sie nun die folgenden drei Local-Search Algorithmen um das TSP zu lösen. Genau wie beim $n$-Damen Problem, kann auch beim TSP eine Lösung als Permutation der Knoten des Graphen gesehen werden. Für das Hill-Climbing und dem Simulated Annealing, besteht eine Aktion daraus, zwei Knoten auf dem Rundweg zu vertauschen. Beim EX3-Algorithmus ist die Aktion unten beschrieben.

**1. Simple Hill-Climbing**

In [ ]:
def tsp_hill_climb(G: nx.Graph) -> int:
  pass

**3. Simulated Annealing**

In [ ]:
def tsp_simulated_annealing(G: nx.Graph, temperature: float) -> int:
  pass

**2. EX3-Algorithmus**

Der EX3-Algorithmus funktioniert ähnlich wie Hill-Climbing, benutzt allerdings eine etwas andere Nachfolgerfunktion. Die Nachfolger eines Rundweges werden bestimmt, indem zunächst zufällig **drei** Kanten aus dem bisherigen Rundweg ausgewählt werden. Dies zerlegt den Rundweg in drei Teilrundwege. Für das erneute zusammenfügen gibt es danach 8 Möglichkeiten (wie im Bild unten zu sehen ist). Der Algorithmus testet nun für jede 3 Kanten alle Nachfolger und wählt denjenigen aus, der die Kosten am stärksten verringert. Dies wird so lange gemacht, bis sich der Rundweg nicht mehr verkürzt.

![EX3 Visualisation](three_opt.png "EX3")

In [ ]:
def tsp_ex3(G: nx.Graph) -> int:
  pass

**Evaluierung**

Vergleichen Sie die drei Algorithmen miteinander. Nutzen Sie dafür die bereitgestellten Benchmarks und die Länge der Optimallösung. Sie sollten Ihre Ergebnisse geeignet visualisieren.